In [ ]:
# model_experiments.ipynb

# Importing libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2, ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Load data
data = np.load("processed_data.npz")
X_train, X_test, y_train, y_test, class_names = (
    data["X_train"],
    data["X_test"],
    data["y_train"],
    data["y_test"],
    data["class_names"]
)

# Data augmentation
datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)
datagen.fit(X_train)

# Experiment 1: MobileNetV2
print("Experiment 1: MobileNetV2")
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(len(class_names), activation="softmax")
])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(datagen.flow(X_train, y_train, batch_size=32), validation_data=(X_test, y_test), epochs=10)

# Plot training results
plt.plot(history.history["accuracy"], label="Training Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.legend()
plt.title("MobileNetV2 Accuracy")
plt.show()

# Save model
model.save("models/experiment_mobilenetv2.h5")

# Experiment 2: ResNet50
print("Experiment 2: ResNet50")
base_model_resnet = ResNet50(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
base_model_resnet.trainable = False

model_resnet = Sequential([
    base_model_resnet,
    GlobalAveragePooling2D(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(len(class_names), activation="softmax")
])

model_resnet.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history_resnet = model_resnet.fit(datagen.flow(X_train, y_train, batch_size=32), validation_data=(X_test, y_test), epochs=10)

# Plot training results
plt.plot(history_resnet.history["accuracy"], label="Training Accuracy")
plt.plot(history_resnet.history["val_accuracy"], label="Validation Accuracy")
plt.legend()
plt.title("ResNet50 Accuracy")
plt.show()

# Save model
model_resnet.save("models/experiment_resnet50.h5")
